In [ ]:
import pandas as pd
train = pd.read_csv('../input/hope-english/english_hope_train.csv',names=['tweet','hope','un'],sep='\t')
val = pd.read_csv('../input/hope-english/english_hope_dev.csv',names=['tweet','hope','un'],sep='\t')
test = pd.read_csv('../input/hope-english/english_hope_test.csv',sep=';')

In [ ]:
train=train[train['hope']!='not-English']
val=val[val['hope']!='not-English']
test=test[test['label']!='not-English']

In [ ]:
from sklearn.preprocessing import LabelEncoder
import torch
encoder=LabelEncoder()
train_df=pd.DataFrame()
valid_df=pd.DataFrame()
test_df=pd.DataFrame()
train_df['text']=train['tweet']
valid_df['text']=val['tweet']
test_df['text']=test['text']
train_df['labels']=encoder.fit_transform(train['hope'])
valid_df['labels']=encoder.fit_transform(val['hope'])
test_df['labels']=encoder.fit_transform(test['label'])

In [ ]:
#focal_loss

!python3 -m pip install -q git+https://github.com/Zhylkaaa/simpletransformers.git@add_losses
!pip install -q tensorboardX
from simpletransformers.config.model_args import ClassificationArgs
from simpletransformers.classification import ClassificationModel
model_args = ClassificationArgs(
    num_train_epochs=5,
    overwrite_output_dir=True, 
    manual_seed=42,
    max_seq_length=160,
    #sliding_window=True,
    eval_batch_size=64,
    do_lower_case=True,
    early_stopping_patience=2,
    save_model_every_epoch=False,
    early_stopping_consider_epochs=True,
    use_early_stopping=True,
    evaluate_during_training=False,
    #wandb_project='Hope_NoOverlap_FL',
    learning_rate=3e-05,
    loss_type = 'focal', # or 'dice' or 'tversky'
    loss_args = {
            'alpha': None,
            'gamma': 2,
            'reduction': 'mean',
            'ignore_index': -100,
        })
model = ClassificationModel(
    model_type='bert',
    model_name='bert-base-multilingual-uncased', 
    use_cuda=torch.cuda.is_available(),
    #cuda_device=3,
    num_labels=len(encoder.classes_), 
    args=model_args)

In [ ]:
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel,ClassificationArgs

In [ ]:
model_args = ClassificationArgs(
    num_train_epochs=3,
    overwrite_output_dir=True, 
    manual_seed=42,
    max_seq_length=160,
    #sliding_window=True,
    eval_batch_size=64,
    do_lower_case=True,
    early_stopping_patience=2,
    save_model_every_epoch=False,
    early_stopping_consider_epochs=True,
    use_early_stopping=True,
    evaluate_during_training=False,
    #wandb_project='ST-Hope-CELoss',
    learning_rate=3e-05)
model = ClassificationModel(
    model_type='bert',
    model_name='bert-base-multilingual-uncased', 
    use_cuda=torch.cuda.is_available(),
    #cuda_device=3,
    num_labels=len(encoder.classes_), 
    args=model_args)

In [ ]:
model.train_model(train_df,args={'train_batch_size':32},eval_df=valid_df)

In [ ]:
val_predictions, raw_outputs = model.predict(valid_df['text'].tolist())
test_predictions, raw_outputs = model.predict(test_df['text'].tolist())

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,f1_score
print('class_report\n',classification_report(valid_df['labels'],val_predictions,target_names=list(encoder.classes_),digits=4))
print('class_report\n',classification_report(test_df['labels'],test_predictions,target_names=list(encoder.classes_),digits=4))

In [ ]:
valid_df['overlap_pred_fl']=val_predictions
test_df['overlap_pred_fl']=test_predictions
valid_df['labels']=encoder.inverse_transform(valid_df['labels'])
valid_df['overlap_pred_fl']=encoder.inverse_transform(valid_df['overlap_pred_fl'])
test_df['labels']=encoder.inverse_transform(test_df['labels'])
test_df['overlap_pred_fl']=encoder.inverse_transform(test_df['overlap_pred_fl'])

In [ ]:
valid_df.to_csv('valid_baseline.csv')
test_df.to_csv('test_baseline.csv')